StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.32.2",
  "_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [2]:
from diffusers import DiffusionPipeline
import torch

def generate_sdxl_image(prompt, negative_prompt="", num_inference_steps=25, guidance_scale=7.5, output_path="generated_image.png"):
    """
    Generates an image using the Stable Diffusion XL (SDXL) model.

    Args:
        prompt (str): The text prompt to generate the image from.
        negative_prompt (str, optional): The negative prompt to avoid unwanted features. Defaults to "".
        num_inference_steps (int, optional): The number of denoising steps. Defaults to 25.
        guidance_scale (float, optional): The guidance scale for classifier-free guidance. Defaults to 7.5.
        output_path (str, optional): The path to save the generated image. Defaults to "generated_image.png".
    """

    try:
        # Load the SDXL pipeline
        pipeline = DiffusionPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16"
        )
        pipeline.to("cuda") # or "cpu"

        # Generate the image
        image = pipeline(prompt=prompt, negative_prompt=negative_prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]

        # Save the image
        image.save(output_path)
        print(f"Image saved to {output_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    prompt = "A majestic lion standing on a rocky cliff overlooking a vast savanna, golden hour, realistic, detailed."
    negative_prompt = "blurry, low quality, distorted, cartoon, anime" # example negative prompt.
    generate_sdxl_image(prompt, negative_prompt=negative_prompt)

    # Example of other parameters
    #generate_sdxl_image(prompt, num_inference_steps=30, guidance_scale=8.0)

/home/bbahl/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/bbahl/miniconda3/envs/torchfeb11/lib/python3.10/site-packages/torch_xla/__init__.py:251: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
2025-03-19 21:18:09.642177: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742419089.868459  170170 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugi

An error occurred: Torch not compiled with CUDA enabled


In [ ]:
from diffusers import DiffusionPipeline
import torch
import torch_xla.core.xla_model as xm
from time import time

model_id = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(model_id)

device = xm.xla_device()
pipe.to(device)

prompt = "A pokemon with green eyes and red legs."
start = time()
inference_steps=2
image = pipe(prompt, num_inference_steps=inference_steps).images[0]
print(f'Compilation time is {time()-start} sec')
image.save("pokemon.png")

start = time()
image = pipe(prompt, num_inference_steps=inference_steps).images[0]
print(f'Inference time is {time()-start} sec after compilation')

/home/bbahl/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/bbahl/miniconda3/envs/torchfeb11/lib/python3.10/site-packages/torch_xla/__init__.py:251: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
2025-03-19 22:12:03.105357: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742422323.121137  178594 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugi